# Background

Links:
- [Python sqlite tutorial](https://towardsdatascience.com/python-has-a-built-in-database-heres-how-to-use-it-47826c10648a)

In [1]:
import sqlite3

In [2]:
# Connects to `movies.db` if it exists; otherwise, creates it
conn = sqlite3.connect('movies.db')
c = conn.cursor()

## Creating tables

In [3]:
def table_exists(table_name):
    c.execute(
        """
        SELECT count(name)
        FROM sqlite_master
        WHERE TYPE = 'table' AND name = '{}'
        """.format(
            table_name
        )
    )
    if c.fetchone()[0] == 1:
        return True
    return False

In [4]:
if not table_exists("movies"):
    c.execute(
        """
        CREATE TABLE movies (
            movie_id INTEGER,
            name TEXT,
            release_year INTEGER,
            genre TEXT,
            rating REAL
        )
        """
    )

In [5]:
c.execute(
    """
    SELECT count(name)
    FROM sqlite_master
    WHERE TYPE = 'table' AND name = '{}'
    """.format(
        "movies"
    )
)
c.fetchone()

(1,)

## Create, Read, Update, Delete 

### Create

In [6]:
def insert_movie(movie_id, name, release_year, genre, rating):
    c.execute(
        """
        INSERT INTO movies (
            movie_id, name, release_year, genre, rating
        ) VALUES (
            ?, ?, ?, ?, ?
        )
        """,
        (movie_id, name, release_year, genre, rating),
    )
    conn.commit()

In [7]:
insert_movie(1, 'Titanic', 1997, 'Drama', 7.8) 
insert_movie(2, 'The Day After Tomorrow', 2004, 'Action', 6.4) 
insert_movie(3, '2012', 2009, 'Action', 5.8) 
insert_movie(4, 'Men in Black', 1997, 'Action', 7.3) 
insert_movie(5, 'World War Z', 2013, 'Romance', 10)

### Read

In [8]:
def get_movies():
    c.execute("""SELECT * FROM movies""")
    data = []
    for row in c.fetchall():
        data.append(row)
    return data

In [9]:
get_movies()

[(1, 'Titanic', 1997, 'Drama', 7.8),
 (2, 'The Day After Tomorrow', 2004, 'Action', 6.4),
 (3, '2012', 2009, 'Action', 5.8),
 (4, 'Men in Black', 1997, 'Action', 7.3),
 (5, 'World War Z', 2013, 'Romance', 10.0)]

In [10]:
def get_movie(movie_id):
    c.execute(
        """
        SELECT *
        FROM movies
        WHERE movie_id = {}
        """.format(
            movie_id
        )
    )
    data = []
    for row in c.fetchall():
        data.append(row)
    return data

In [11]:
get_movie(2)

[(2, 'The Day After Tomorrow', 2004, 'Action', 6.4)]

### Update

In [12]:
def update_movie(movie_id, update_dict):
    valid_keys = ["name", "release_year", "genre", "rating"]
    for key in update_dict.keys():
        if key not in valid_keys:
            raise Exception("Invalid field name!")
            
    for key in update_dict.keys():
        if type(update_dict[key]) == str:
            stmt = """
                UPDATE movies
                SET {} = '{}'
                WHERE movie_id = {}
            """.format(
                key, update_dict[key], movie_id
            )
        else:
            stmt = """
                UPDATE movies
                SET {} = {}
                WHERE movie_id = {}
            """.format(
                key, update_dict[key], movie_id
            )
        
        c.execute(stmt)
        conn.commit()

In [13]:
print(get_movie(5))
update_movie(5, {'genre': 'Horror', 'rating': 7.0})
print(get_movie(5))

[(5, 'World War Z', 2013, 'Romance', 10.0)]
[(5, 'World War Z', 2013, 'Horror', 7.0)]


### Delete

In [14]:
def delete_movie(movie_id):
    c.execute(
        """
        DELETE FROM movies
        WHERE movie_id = {}
        """.format(
            movie_id
        )
    )
    conn.commit()

In [15]:
delete_movie(3)
get_movies()

[(1, 'Titanic', 1997, 'Drama', 7.8),
 (2, 'The Day After Tomorrow', 2004, 'Action', 6.4),
 (4, 'Men in Black', 1997, 'Action', 7.3),
 (5, 'World War Z', 2013, 'Horror', 7.0)]

In [16]:
for n in range(1, 6):
    delete_movie(n)